In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.9.1'

    Better performance with the tf.data API 
https://www.tensorflow.org/guide/data_performance

# Measuring the performence using pretetch

In [3]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples): # reading file
        # Opening the file
        time.sleep(0.03) # file opening time

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015) # delay time to read each line

            yield (sample_idx,) #yield is a generator | it is a dummy class | return the particular sample index | return each line

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [4]:


def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01) # training time

    | The naive approach

In [5]:

%%timeit
benchmark(FileDataset())

638 ms ± 72.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


    | The Prefetch API approach

In [6]:
%%timeit
benchmark(FileDataset().prefetch(3))

553 ms ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


    | tf.data.AUTOTUNE, 

which will prompt the tf.data runtime to tune the value dynamically at runtime.

In [7]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

595 ms ± 185 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Cache API

In [8]:
dataset = tf.data.Dataset.range(5)
for i in dataset:
    print(i.numpy())

0
1
2
3
4


In [9]:
dataset=dataset.map(lambda x: x**2)
for i in dataset:
    print(i.numpy())

0
1
4
9
16


In [10]:
dataset=dataset.cache()
for data in dataset.as_numpy_iterator():
    print(data) # or

0
1
4
9
16


In [11]:
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [12]:
# OR

dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [13]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [14]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [15]:
def  mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ()) # it is introducing some kinds of delay
    return s

    |     | The naive approach

In [16]:
%%timeit -r1 -n1 # -r1 ,-n1 means just run the one loop
benchmark(FileDataset().map(mapped_function), 5)

2.33 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


    | Using Cache 

2nd, 3rd..... epoch don't running  mapped_function()

In [17]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

779 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
